In [1]:
import os
os.system("pip install -q wget")

0

In [ ]:
import wget
import tarfile
import torchaudio
import pandas as pd
from huggingface_hub import snapshot_download, login
login("<your_huggingface_token>")

In [3]:
os.chdir("/content")

In [ ]:
from huggingface_hub import HfApi
import os
api = HfApi()
repo_id = "heboya8/t5-tts-temp-model"
save_path = "/content/"
os.makedirs(save_path, exist_ok=True)

# Download the dataset
snapshot_download(repo_id=repo_id, repo_type="model", local_dir=save_path)

In [ ]:
!rm -rf 45


In [ ]:
!pwd

In [ ]:
!ls -a /content/F5-TTS

In [ ]:
# Step 1: Set Up the Environment
os.system("pip install -e . >/dev/null 2>&1")
os.system("pip install torch==2.4.0+cu124 torchaudio==2.4.0+cu124 torchvision==0.19.0+cu124 --extra-index-url https://download.pytorch.org/whl/cu124 >/dev/null 2>&1")
os.system("pip install accelerate==0.33.0 tensorboard >/dev/null 2>&1")
if not os.path.exists("F5-TTS"):
    os.system("git clone https://github.com/SWivid/F5-TTS.git")
os.chdir("F5-TTS")
os.system("pip install -e . >/dev/null 2>&1")


Cloning into 'F5-TTS'...


In [ ]:
os.chdir("/content/F5-TTS")
# os.chdir("F5-TTS-Vietnamese")

In [ ]:
!pwd

In [ ]:
!mkdir ./ckpts/vin100h-preprocessed-v2
# !cp -r /kaggle/input/vi-fine-tuned-t5-tts/22/model_last.pt \
# ./ckpts/vin100h-preprocessed-v2/

In [ ]:
!cp -r /kaggle/input/vi-fine-tuned-t5-tts/67/model_last.pt ./ckpts/vin100h-preprocessed-v2/

In [9]:
!ls -a ./ckpts/vin100h-preprocessed-v2

.  ..  model_last.pt


In [ ]:
import json
import os
from pathlib import Path
import shutil
import torchaudio
from datasets import load_dataset
from datasets.arrow_writer import ArrowWriter
from tqdm import tqdm
import soundfile as sf
import csv

def save_dataset_to_local_disk(output_dir="./data/vin100h-preprocessed-v2",
                               base_model="htdung167/vin100h-preprocessed-v2",
                               audio_header='audio', text_header='transcription'):
  
    wavs_dir = os.path.join(output_dir, "wavs")
    metadata_path = os.path.join(output_dir, "metadata.csv")
    os.makedirs(wavs_dir, exist_ok=True)

    ds = load_dataset(base_model)['train']
    metadata = []

    for idx, sample in tqdm(enumerate(ds), total=len(ds),
                            desc="Saving samples to directory"):
        audio_array = sample[audio_header]['array']
        sampling_rate = sample[audio_header]['sampling_rate']
        filename = f"audio_{idx:06d}.wav"
        sf.write(os.path.join(wavs_dir, filename), audio_array, sampling_rate)
        # metadata.append([f"wavs/{filename}", sample['preprocessed_sentence_v2']])
        metadata.append([f"wavs/{filename}", sample[text_header]])
        # metadata.append([f"{filename}", sample['transcription']])
        
    with open(metadata_path, 'w', newline='', encoding='utf-8') as f:
        csv.writer(f, delimiter='|').writerows(metadata)

    print(f"Dataset saved to {output_dir}")


In [ ]:
output_dir = "./data/vin100h-preprocessed-v2"
tokenizer_type = "pinyin"

save_dataset_to_local_disk(output_dir=output_dir,
                           base_model="htdung167/vin100h-preprocessed-v2",
                           text_header="preprocessed_sentence_v2"
                          )

In [ ]:
!python ./src/f5_tts/train/datasets/prepare_csv_wavs.py \
    "./data/vin100h-preprocessed-v2" \
    "./data/vin100h-preprocessed-v2_pinyin" \
    --workers 4 # Sets the number of parallel processes for preprocessing.

In [10]:
%%writefile ./src/f5_tts/configs/vi-fine-tuned-t5-tts.yaml
hydra:
  run:
    dir: ckpts/${model.name}_${model.mel_spec.mel_spec_type}_${model.tokenizer}_${datasets.name}/${now:%Y-%m-%d}/${now:%H-%M-%S}

datasets:
  name: vin100h-preprocessed-v2  # dataset name
  batch_size_per_gpu: 3200  # 1 GPUs, 1 * 3200 = 3200
  batch_size_type: frame  # frame | sample
  max_samples: 64  # max sequences per batch if use frame-wise batch_size. we set 32 for small models, 64 for base models
  num_workers: 4

optim:
  epochs: 10
  learning_rate: 1e-5
  num_warmup_updates: 2761  # warmup updates
  grad_accumulation_steps: 2  # note: updates = steps / grad_accumulation_steps
  max_grad_norm: 1.0  # gradient clipping
  bnb_optimizer: False  # use bnb 8bit AdamW optimizer or not

model:
  name: vi_fine_tuned_t5_tts  # model name
  tokenizer: pinyin  # tokenizer type
  tokenizer_path: null  # if 'custom' tokenizer, define the path want to use (should be vocab.txt)
  backbone: DiT
  arch:
    dim: 1024
    depth: 22
    heads: 16
    ff_mult: 2
    text_dim: 512
    text_mask_padding: False
    conv_layers: 4
    pe_attn_head: 1
    checkpoint_activations: False  # recompute activations and save memory for extra compute
  mel_spec:
    target_sample_rate: 24000
    n_mel_channels: 100
    hop_length: 256
    win_length: 1024
    n_fft: 1024
    mel_spec_type: vocos  # vocos | bigvgan
  vocoder:
    is_local: False  # use local offline ckpt or not
    local_path: null  # local vocoder path

ckpts:
  logger: null  # wandb | tensorboard | null
  log_samples: True  # infer random sample per save checkpoint. wip, normal to fail with extra long samples
  save_per_updates: 4000  # save checkpoint per updates
  keep_last_n_checkpoints: 1  # -1 to keep all, 0 to not save intermediate, > 0 to keep last N checkpoints
  last_per_updates: 4000  # save last checkpoint per updates
  save_dir: ckpts/${model.name}_${model.mel_spec.mel_spec_type}_${model.tokenizer}_${datasets.name}

Writing ./src/f5_tts/configs/vi-fine-tuned-t5-tts.yaml


In [11]:
!accelerate config default

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [13]:
!echo go

go


In [14]:
!pwd

/content/F5-TTS


In [14]:
# ************
!accelerate launch ./src/f5_tts/train/finetune_cli.py \
                    --exp_name F5TTS_Base \
                    --dataset_name vin100h-preprocessed-v2 \
                    --finetune \
                    --tokenizer pinyin \
                    --learning_rate 1e-05 \
                    --batch_size_type frame \
                    --batch_size_per_gpu 3200 \
                    --max_samples 64 \
                    --grad_accumulation_steps 2 \
                    --max_grad_norm 1 \
                    --epochs 69 \
                    --num_warmup_updates 2761 \
                    --save_per_updates 4000 \
                    --keep_last_n_checkpoints 1 \
                    --last_per_updates 4000 \
                    --log_samples \
                    --pretrain ./ckpts/vin100h-preprocessed-v2/model_last.pt
                    # [--pretrain PRETRAIN
                    #10 6182


Epoch 69/69:  97%|▉| 6010/6182 [3:21:35<05:35,  1.95s/update, loss=0.613, update
Epoch 69/69:  97%|▉| 6010/6182 [3:21:35<05:35,  1.95s/update, loss=0.629, update
Epoch 69/69:  97%|▉| 6011/6182 [3:21:37<05:17,  1.86s/update, loss=0.629, update
Epoch 69/69:  97%|▉| 6011/6182 [3:21:37<05:17,  1.86s/update, loss=0.688, update
Epoch 69/69:  97%|▉| 6012/6182 [3:21:39<05:23,  1.90s/update, loss=0.688, update
Epoch 69/69:  97%|▉| 6012/6182 [3:21:39<05:23,  1.90s/update, loss=0.433, update
Epoch 69/69:  97%|▉| 6013/6182 [3:21:41<05:33,  1.97s/update, loss=0.433, update
Epoch 69/69:  97%|▉| 6013/6182 [3:21:41<05:33,  1.97s/update, loss=0.586, update
Epoch 69/69:  97%|▉| 6014/6182 [3:21:43<05:19,  1.90s/update, loss=0.586, update
Epoch 69/69:  97%|▉| 6014/6182 [3:21:43<05:19,  1.90s/update, loss=0.69, update=
Epoch 69/69:  97%|▉| 6015/6182 [3:21:45<05:18,  1.90s/update, loss=0.69, update=
Epoch 69/69:  97%|▉| 6015/6182 [3:21:45<05:18,  1.90s/update, loss=0.744, update
Epoch 69/69:  97%|▉| 6016/6

#### ok

In [15]:
!echo abc

abc


In [18]:
!echo done

done


# Copy and save

In [16]:
# *******************88888
saving_path = '/kaggle/working/69'
os.makedirs(saving_path, exist_ok=True)
!cp -r ./ckpts/vin100h-preprocessed-v2/model_last.pt $saving_path

In [17]:
# Upload the folder to the repository root
api.upload_large_folder(
    folder_path="/kaggle/working/",  # Local folder path
    repo_id="heboya8/t5-tts-temp-model",
    repo_type="model",
    # multi_commits=True,  # Enable resumable uploads
    # multi_commits_verbose=True  # Show progress
)

Recovering from metadata files:   0%|          | 0/1 [00:00<?, ?it/s]




---------- 2025-06-13 17:29:03 (0:00:00) ----------
Files:   hashed 0/1 (0.0/5.4G) | pre-uploaded: 0/0 (0.0/5.4G) (+1 unsure) | committed: 0/1 (0.0/5.4G) | ignored: 0
Workers: hashing: 1 | get upload mode: 0 | pre-uploading: 0 | committing: 0 | waiting: 1
---------------------------------------------------


model_last.pt:   0%|          | 0.00/5.39G [00:00<?, ?B/s]


---------- 2025-06-13 17:30:03 (0:01:00) ----------
Files:   hashed 1/1 (5.4G/5.4G) | pre-uploaded: 0/1 (0.0/5.4G) | committed: 0/1 (0.0/5.4G) | ignored: 0
Workers: hashing: 0 | get upload mode: 0 | pre-uploading: 1 | committing: 0 | waiting: 1
---------------------------------------------------
                                           
---------- 2025-06-13 17:31:03 (0:02:00) ----------
Files:   hashed 1/1 (5.4G/5.4G) | pre-uploaded: 0/1 (0.0/5.4G) | committed: 0/1 (0.0/5.4G) | ignored: 0
Workers: hashing: 0 | get upload mode: 0 | pre-uploading: 1 | committing: 0 | waiting: 1
---------------------------------------------------
                                           
---------- 2025-06-13 17:32:03 (0:03:00) ----------
Files:   hashed 1/1 (5.4G/5.4G) | pre-uploaded: 0/1 (0.0/5.4G) | committed: 0/1 (0.0/5.4G) | ignored: 0
Workers: hashing: 0 | get upload mode: 0 | pre-uploading: 1 | committing: 0 | waiting: 1
---------------------------------------------------
                    

# Prune Checkpoint

In [ ]:
import torch

def prune_checkpoint(checkpoint_path: str, new_checkpoint_path: str, save_ema: bool, safetensors: bool) -> str:
    try:
        checkpoint = torch.load(checkpoint_path, weights_only=True)
        print("Original Checkpoint Keys:", checkpoint.keys())

        to_retain = "ema_model_state_dict" if save_ema else "model_state_dict"
        try:
            model_state_dict_to_retain = checkpoint[to_retain]
        except KeyError:
            return f"{to_retain} not found in the checkpoint."

        if safetensors:
            new_checkpoint_path = new_checkpoint_path.replace(".pt", ".safetensors")
            save_file(model_state_dict_to_retain, new_checkpoint_path)
        else:
            new_checkpoint_path = new_checkpoint_path.replace(".safetensors", ".pt")
            new_checkpoint = {"ema_model_state_dict": model_state_dict_to_retain}
            torch.save(new_checkpoint, new_checkpoint_path)

        return f"New checkpoint saved at: {new_checkpoint_path}"

    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
# Prune a checkpoint after training resize model
result = prune_checkpoint(
    checkpoint_path="/kaggle/working/F5-TTS/ckpts/vin100h-preprocessed-v2/model_last.pt",
    new_checkpoint_path="/root/.cache/abc.pt",
    save_ema=False,
    safetensors=False
)
print(result)

# Inference

In [ ]:
from IPython.display import Audio
Audio(filename="./data/vin100h-preprocessed-v2/wavs/audio_000010.wav")

In [ ]:
!python ./src/f5_tts/infer/infer_cli.py \
        --model "vin100h-preprocessed-v2" \
        --model_cfg "./src/f5_tts/configs/F5TTS_Base.yaml" \
        --ckpt_file "./ckpts/vin100h-preprocessed-v2/model_last.pt" \
        --vocab_file "./data/vin100h-preprocessed-v2_pinyin/vocab.txt" \
        --ref_audio "./data/vin100h-preprocessed-v2/wavs/audio_000010.wav" \
        --ref_text "Về giá cả so với giá bán ngoài các siêu thị thì dâu trái ở đây rẻ hơn khá nhiều. Giả sử như bó rau ở siêu thị bán khoảng 2 đô la một bó thì ở đây chỉ có một đô la một bó. Có khi mua 50 bó được tặng thêm một bó nữa." \
        --gen_text "Chào cả nhà cả nhà khỏe không" \
        --output_dir "/kaggle/working"
        # --output_file "/content/abc.wav"

In [ ]:
from IPython.display import Audio
Audio(filename="/kaggle/working/infer_cli_basic.wav")

## Download

In [12]:
from huggingface_hub import HfApi
from huggingface_hub import snapshot_download
import os
api = HfApi()
!git lfs install --force

# Define the dataset name and local directory
repo_id = "heboya8/facebook-musicgen-small"
save_path = "/root/.cache"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Download the dataset
snapshot_download(repo_id=repo_id, repo_type="dataset", local_dir=save_path)

Updated git hooks.
Git LFS initialized.


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

data_compress.zip:   0%|          | 0.00/10.1G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/2.46k [00:00<?, ?B/s]

'/root/.cache'

In [13]:
!unzip -q -o /root/.cache/data_compress.zip -d "."

In [ ]:
!ls -a data ./ckpts/vin100h-preprocessed-v2

In [ ]:
os.makedirs("./ckpts/vin100h-preprocessed-v2", exist_ok=True)
!cp -r /kaggle/input/vi-fine-tuned-t5-tts/5/* ./ckpts/vin100h-preprocessed-v2/

## Upload

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import snapshot_download
# Initialize API
api = HfApi()

# Upload the folder to the repository root
api.upload_large_folder(
    folder_path="/root/.cache/dataset",  # Local folder path
    repo_id="heboya8/f5-tts-dataset",
    repo_type="dataset",
    # multi_commits=True,  # Enable resumable uploads
    # multi_commits_verbose=True  # Show progress
)

In [ ]:
# Upload the folder to the repository root
api.upload_large_folder(
    folder_path="/kaggle/working/",  # Local folder path
    repo_id="heboya8/t5-tts-temp-model",
    repo_type="model",
    # multi_commits=True,  # Enable resumable uploads
    # multi_commits_verbose=True  # Show progress
)

## /kaggle/working/F5-TTS/ckpts/vin100h-preprocessed-v2/model_last.ptDowload Dataset

In [ ]:
!mkdir dataset

In [ ]:
!rm -rf d /root/.cache/dataset

In [ ]:
!unzip -q /kaggle/working/F5-TTS/~/.cache/data_compress.zip -d /root/.cache/dataset